In [ ]:
! pip install kaggle


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

! cp /content/drive/MyDrive/kaggle.json ~/
! mkdir ~/.kaggle
! mv ~/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json

Mounted at /content/drive


In [ ]:
! kaggle datasets download -d eldarsultanow/bsd-dataset-expanded-around-original

100% 1.51G/1.51G [00:19<00:00, 98.8MB/s]
100% 1.51G/1.51G [00:19<00:00, 83.4MB/s]


In [ ]:
! unzip bsd-dataset-expanded-around-original.zip

Archive:  bsd-dataset-expanded-around-original.zip
  inflating: expanded_dataset.csv    


In [ ]:
import pandas as pd
import numpy as np
import sympy
from scipy.interpolate import interp1d
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.ensemble import GradientBoostingRegressor
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping
from sympy import prime, isprime, primerange
import random
import math
import fileinput
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, f1_score


In [ ]:
import pandas as pd

df = pd.read_csv("expanded_dataset.csv", index_col=False, header=None, skiprows=lambda i: i % 5 != 0)


In [ ]:
import pandas as pd

# List of indices to read
indices_to_read = [1, 0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 18,
                   21, 20, 26, 25, 23, 30, 28, 27, 22, 24, 35, 31, 29, 42, 41, 37, 34, 33,
                   49, 45, 46, 32, 52, 56, 44, 36, 51, 58, 39, 57, 47, 50, 69, 43, 40, 53,
                   59, 60, 64, 38, 67, 55, 125, 82, 138, 341, 1160, 368, 435, 436, 437,
                   425, 434, 433, 432, 431, 430, 429, 428, 427, 438, 426, 410, 424, 423,
                   422, 421, 420, 419, 418, 417, 416, 415, 414, 413, 412, 411, 440, 439,
                   468]

df = pd.read_csv("expanded_dataset.csv", index_col=False, header=None, usecols=indices_to_read,skiprows=1)

print(df)


        0           1     2     3     4     5     6     7     8     9     ...  \
0          1  7578874153    -1     1     0     0    -2     6    -4     0  ...   
1          1  7526030717    -1     0     2     0    -1     0     2     3  ...   
2          1  7078096709    -1     1     0    -3     2    -4    -6    -1  ...   
3          0  7886103067     1    -1     0     0     5    -1     2    -4  ...   
4          1  7124778971    -1    -1    -2     2    -1     0    -2    -4  ...   
...      ...         ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   
959994     3  8130625091    -1     0    -3    -2     0    -3    -1     0  ...   
959995     3  8278561751    -1    -1     1    -3     1    -4     4     6  ...   
959996     3  8919340363    -1     0    -2     0    -5    -1     4     0  ...   
959997     3  8375056733     1     1    -2     2    -3     0     1    -1  ...   
959998     3  8213233843     1    -1    -1     2    -4    -3    -5    -2  ...   

        433   434   435   4

In [ ]:
new_columns = ['rank', 'con', 'root'] + [f'ap_{i}' for i in range(1, 98)]

df.columns = new_columns

In [ ]:
print(df)

        rank         con  root  ap_1  ap_2  ap_3  ap_4  ap_5  ap_6  ap_7  ...  \
0          1  7578874153    -1     1     0     0    -2     6    -4     0  ...   
1          1  7526030717    -1     0     2     0    -1     0     2     3  ...   
2          1  7078096709    -1     1     0    -3     2    -4    -6    -1  ...   
3          0  7886103067     1    -1     0     0     5    -1     2    -4  ...   
4          1  7124778971    -1    -1    -2     2    -1     0    -2    -4  ...   
...      ...         ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   
959994     3  8130625091    -1     0    -3    -2     0    -3    -1     0  ...   
959995     3  8278561751    -1    -1     1    -3     1    -4     4     6  ...   
959996     3  8919340363    -1     0    -2     0    -5    -1     4     0  ...   
959997     3  8375056733     1     1    -2     2    -3     0     1    -1  ...   
959998     3  8213233843     1    -1    -1     2    -4    -3    -5    -2  ...   

        ap_88  ap_89  ap_90

In [ ]:
df['rank'].value_counts()

1    332709
0    312290
2    210000
3    103616
4      1335
5        48
6         1
Name: rank, dtype: int64

In [ ]:
df_2 = df.loc[df['rank'] == 2].sample(n=30000, random_state=1)
df_3 = df.loc[df['rank'] == 3].sample(n=30000, random_state=1)
df_3_2 = pd.concat([df_2, df_3])
df_1 = df.loc[df['rank'] == 1].sample(n=30000, random_state=1)
df_0 = df.loc[df['rank'] == 0].sample(n=30000, random_state=1)
df_0_1 = pd.concat([df_1, df_0])

df_sub = pd.concat([df_0_1, df_3_2])
df_sub['rank'].value_counts()

1    30000
0    30000
2    30000
3    30000
Name: rank, dtype: int64

In [ ]:
df_sub = df_sub[~df_sub['rank'].isin([4,5,6])]

In [ ]:
for i in range(1, 97):
    ap_col = f"ap_{i+1}"
    df_sub[ap_col].dropna(inplace=True)
    if (df_sub[ap_col] == 0).any():
        df_sub[ap_col].replace(0, df_sub[ap_col].mean(), inplace=True)

In [ ]:
feature_col = [col for col in df.columns if col.startswith('ap_')]
X = df_sub[ feature_col]
y = df_sub['rank']

In [ ]:
gbm = GradientBoostingRegressor(n_estimators=100, max_depth=3, learning_rate=0.1)
gbm.fit(X, y)

GradientBoostingRegressor()

In [ ]:
importances = gbm.feature_importances_
top_100_indices = importances.argsort()[::-1][:100]
print(top_100_indices)

[   1    0    2    3    4    5    6    7    8    9   10   11   12   13
   14   15   16   17   19   18   21   20   26   25   23   30   28   27
   22   24   35   31   29   42   41   37   34   33   49   45   46   32
   52   56   44   36   51   58   39   57   47   50   69   43   40   53
   59   60   64   38   67   55  125   82  138  341 1160  368  435  436
  437  425  434  433  432  431  430  429  428  427  438  426  410  424
  423  422  421  420  419  418  417  416  415  414  413  412  411  440
  439  468]


In [ ]:
importances = gbm.feature_importances_
df_columns = feature_col
top_100_indices = importances.argsort()[::-1][:500]

# Get the top 100 'ap' column names
top_100_columns = [df_columns[i] for i in top_100_indices if df_columns[i].startswith('ap_')]

print(top_100_columns)

['ap_2', 'ap_1', 'ap_3', 'ap_4', 'ap_5', 'ap_6', 'ap_7', 'ap_8', 'ap_9', 'ap_10', 'ap_11', 'ap_12', 'ap_13', 'ap_14', 'ap_15', 'ap_16', 'ap_17', 'ap_18', 'ap_20', 'ap_19', 'ap_22', 'ap_21', 'ap_27', 'ap_26', 'ap_24', 'ap_31', 'ap_29', 'ap_28', 'ap_23', 'ap_25', 'ap_36', 'ap_32', 'ap_30', 'ap_43', 'ap_42', 'ap_38', 'ap_35', 'ap_34', 'ap_50', 'ap_46', 'ap_47', 'ap_33', 'ap_53', 'ap_57', 'ap_45', 'ap_37', 'ap_52', 'ap_59', 'ap_40', 'ap_58', 'ap_48', 'ap_51', 'ap_70', 'ap_44', 'ap_41', 'ap_54', 'ap_60', 'ap_61', 'ap_65', 'ap_39', 'ap_68', 'ap_56', 'ap_126', 'ap_83', 'ap_139', 'ap_342', 'ap_1161', 'ap_369', 'ap_436', 'ap_437', 'ap_438', 'ap_426', 'ap_435', 'ap_434', 'ap_433', 'ap_432', 'ap_431', 'ap_430', 'ap_429', 'ap_428', 'ap_439', 'ap_427', 'ap_411', 'ap_425', 'ap_424', 'ap_423', 'ap_422', 'ap_421', 'ap_420', 'ap_419', 'ap_418', 'ap_417', 'ap_416', 'ap_415', 'ap_414', 'ap_413', 'ap_412', 'ap_441', 'ap_440', 'ap_469', 'ap_442', 'ap_467', 'ap_461', 'ap_462', 'ap_463', 'ap_464', 'ap_465', 

In [ ]:
top_features = ['ap_2', 'ap_1', 'ap_3', 'ap_4', 'ap_5', 'ap_6', 'ap_7', 'ap_8', 'ap_9', 'ap_10', 'ap_11', 'ap_12', 'ap_13', 'ap_14', 'ap_15', 'ap_16', 'ap_17', 'ap_18', 'ap_20', 'ap_19', 'ap_22', 'ap_21', 'ap_27', 'ap_26', 'ap_24', 'ap_31', 'ap_29', 'ap_28', 'ap_23', 'ap_25', 'ap_36', 'ap_32', 'ap_30', 'ap_43', 'ap_42', 'ap_38', 'ap_35', 'ap_34', 'ap_50', 'ap_46', 'ap_47', 'ap_33', 'ap_53', 'ap_57', 'ap_45', 'ap_37', 'ap_52', 'ap_59', 'ap_40', 'ap_58', 'ap_48', 'ap_51', 'ap_70', 'ap_44', 'ap_41', 'ap_54', 'ap_60', 'ap_61', 'ap_65', 'ap_39', 'ap_68', 'ap_56', 'ap_126', 'ap_83', 'ap_139', 'ap_342', 'ap_1161', 'ap_369', 'ap_436', 'ap_437', 'ap_438', 'ap_426', 'ap_435', 'ap_434', 'ap_433', 'ap_432', 'ap_431', 'ap_430', 'ap_429', 'ap_428', 'ap_439', 'ap_427', 'ap_411', 'ap_425', 'ap_424', 'ap_423', 'ap_422', 'ap_421', 'ap_420', 'ap_419', 'ap_418', 'ap_417', 'ap_416', 'ap_415', 'ap_414', 'ap_413', 'ap_412', 'ap_441', 'ap_440', 'ap_469']

primes = [-301/303, -299/303, -99/101, -295/303, -293/303, -97/101, -289/303, -287/303, -95/101, -283/303, -281/303, -93/101, -277/303, -275/303, -91/101, -271/303, -269/303, -89/101, -265/303, -263/303, -87/101, -259/303, -257/303, -85/101, -253/303, -251/303, -83/101, -247/303, -245/303, -81/101, -241/303, -239/303, -79/101, -235/303, -233/303, -77/101, -229/303, -227/303, -75/101, -223/303, -221/303, -73/101, -217/303, -215/303, -71/101, -211/303, -209/303, -69/101, -205/303, -203/303, -67/101, -199/303, -197/303, -65/101, -193/303, -191/303, -63/101, -187/303, -185/303, -61/101, -181/303, -179/303, -59/101, -175/303, -173/303, -57/101, -169/303, -167/303, -55/101, -163/303, -161/303, -53/101, -157/303, -155/303, -51/101, -151/303, -149/303, -49/101, -145/303, -143/303, -47/101, -139/303, -137/303, -45/101, -133/303, -131/303, -43/101, -127/303, -125/303, -41/101, -121/303, -119/303, -39/101, -115/303, -113/303, -37/101, -109/303, -107/303, -35/101, -103/303, -1/3, -33/101, -97/303, -95/303, -31/101, -91/303, -89/303, -29/101, -85/303, -83/303, -27/101, -79/303, -77/303, -25/101, -73/303, -71/303, -23/101, -67/303, -65/303, -21/101, -61/303, -59/303, -19/101, -55/303, -53/303, -17/101, -49/303, -47/303, -15/101, -43/303, -41/303, -13/101, -37/303, -35/303, -11/101, -31/303, -29/303, -9/101, -25/303, -23/303, -7/101, -19/303, -17/303, -5/101, -13/303, -11/303, -3/101, -7/303, -5/303, -1/101, -1/303, 1/303, 1/101, 5/303, 7/303, 3/101, 11/303, 13/303, 5/101, 17/303, 19/303, 7/101, 23/303, 25/303, 9/101, 29/303, 31/303, 11/101, 35/303, 37/303, 13/101, 41/303, 43/303, 15/101, 47/303, 49/303, 17/101, 53/303, 55/303, 19/101, 59/303, 61/303, 21/101, 65/303, 67/303, 23/101, 71/303, 73/303, 25/101, 77/303, 79/303, 27/101, 83/303, 85/303, 29/101, 89/303, 91/303, 31/101, 95/303, 97/303, 33/101, 1/3, 103/303, 35/101, 107/303, 109/303, 37/101, 113/303, 115/303, 39/101, 119/303, 121/303, 41/101, 125/303, 127/303, 43/101, 131/303, 133/303, 45/101, 137/303, 139/303, 47/101, 143/303, 145/303, 49/101, 149/303, 151/303, 51/101, 155/303, 157/303, 53/101, 161/303, 163/303, 55/101, 167/303, 169/303, 57/101, 173/303, 175/303, 59/101, 179/303, 181/303, 61/101, 185/303, 187/303, 63/101, 191/303, 193/303, 65/101, 197/303, 199/303, 67/101, 203/303, 205/303, 69/101, 209/303, 211/303, 71/101, 215/303, 217/303, 73/101, 221/303, 223/303, 75/101, 227/303, 229/303, 77/101, 233/303, 235/303, 79/101, 239/303, 241/303, 81/101, 245/303, 247/303, 83/101, 251/303, 253/303, 85/101, 257/303, 259/303, 87/101, 263/303, 265/303, 89/101, 269/303, 271/303, 91/101, 275/303, 277/303, 93/101, 281/303, 283/303, 95/101, 287/303, 289/303, 97/101, 293/303, 295/303, 99/101, 299/303, 301/303, 1]
abs_prime = [abs(prime) for prime in primes]

In [ ]:
top_features_200 = ['ap_2', 'ap_1', 'ap_3', 'ap_4', 'ap_5', 'ap_6', 'ap_7', 'ap_8', 'ap_9', 'ap_10', 'ap_11', 'ap_12', 'ap_13', 'ap_14', 'ap_15', 'ap_16', 'ap_17', 'ap_18', 'ap_20', 'ap_19', 'ap_22', 'ap_21', 'ap_27', 'ap_26', 'ap_24', 'ap_31', 'ap_29', 'ap_28', 'ap_23', 'ap_25', 'ap_36', 'ap_32', 'ap_30', 'ap_43', 'ap_42', 'ap_38', 'ap_35', 'ap_34', 'ap_50', 'ap_46', 'ap_47', 'ap_33', 'ap_53', 'ap_57', 'ap_45', 'ap_37', 'ap_52', 'ap_59', 'ap_40', 'ap_58', 'ap_48', 'ap_51', 'ap_70', 'ap_44', 'ap_41', 'ap_54', 'ap_60', 'ap_61', 'ap_65', 'ap_39', 'ap_68', 'ap_56', 'ap_126', 'ap_83', 'ap_139', 'ap_342', 'ap_1161', 'ap_369', 'ap_436', 'ap_437', 'ap_438', 'ap_426', 'ap_435', 'ap_434', 'ap_433', 'ap_432', 'ap_431', 'ap_430', 'ap_429', 'ap_428', 'ap_439', 'ap_427', 'ap_411', 'ap_425', 'ap_424', 'ap_423', 'ap_422', 'ap_421', 'ap_420', 'ap_419', 'ap_418', 'ap_417', 'ap_416', 'ap_415', 'ap_414', 'ap_413', 'ap_412', 'ap_441', 'ap_440', 'ap_469', 'ap_442', 'ap_467', 'ap_461', 'ap_462', 'ap_463', 'ap_464', 'ap_465', 'ap_466', 'ap_468', 'ap_443', 'ap_409', 'ap_470', 'ap_471', 'ap_472', 'ap_473', 'ap_474', 'ap_460', 'ap_459', 'ap_458', 'ap_457', 'ap_456', 'ap_455', 'ap_454', 'ap_453', 'ap_452', 'ap_451', 'ap_450', 'ap_449', 'ap_448', 'ap_447', 'ap_446', 'ap_445', 'ap_444', 'ap_410', 'ap_377', 'ap_408', 'ap_357', 'ap_372', 'ap_371', 'ap_370', 'ap_368', 'ap_367', 'ap_366', 'ap_365', 'ap_364', 'ap_363', 'ap_362', 'ap_361', 'ap_360', 'ap_359', 'ap_358', 'ap_356', 'ap_374', 'ap_355', 'ap_354', 'ap_353', 'ap_352', 'ap_351', 'ap_350', 'ap_349', 'ap_348', 'ap_347', 'ap_346', 'ap_345', 'ap_344', 'ap_343', 'ap_341', 'ap_373', 'ap_375', 'ap_407', 'ap_392', 'ap_406', 'ap_405', 'ap_404', 'ap_403', 'ap_402', 'ap_401', 'ap_400', 'ap_399', 'ap_398', 'ap_397', 'ap_396', 'ap_395', 'ap_394', 'ap_393', 'ap_391', 'ap_376', 'ap_390', 'ap_389', 'ap_388', 'ap_387', 'ap_386', 'ap_385', 'ap_384', 'ap_383', 'ap_382', 'ap_381', 'ap_380', 'ap_379', 'ap_378']


In [ ]:
top_features_300 =['ap_2', 'ap_1', 'ap_3', 'ap_4', 'ap_5', 'ap_6', 'ap_7', 'ap_8', 'ap_9', 'ap_10', 'ap_11', 'ap_12', 'ap_13', 'ap_14', 'ap_15', 'ap_16', 'ap_17', 'ap_18', 'ap_20', 'ap_19', 'ap_22', 'ap_21', 'ap_27', 'ap_26', 'ap_24', 'ap_31', 'ap_29', 'ap_28', 'ap_23', 'ap_25', 'ap_36', 'ap_32', 'ap_30', 'ap_43', 'ap_42', 'ap_38', 'ap_35', 'ap_34', 'ap_50', 'ap_46', 'ap_47', 'ap_33', 'ap_53', 'ap_57', 'ap_45', 'ap_37', 'ap_52', 'ap_59', 'ap_40', 'ap_58', 'ap_48', 'ap_51', 'ap_70', 'ap_44', 'ap_41', 'ap_54', 'ap_60', 'ap_61', 'ap_65', 'ap_39', 'ap_68', 'ap_56', 'ap_126', 'ap_83', 'ap_139', 'ap_342', 'ap_1161', 'ap_369', 'ap_436', 'ap_437', 'ap_438', 'ap_426', 'ap_435', 'ap_434', 'ap_433', 'ap_432', 'ap_431', 'ap_430', 'ap_429', 'ap_428', 'ap_439', 'ap_427', 'ap_411', 'ap_425', 'ap_424', 'ap_423', 'ap_422', 'ap_421', 'ap_420', 'ap_419', 'ap_418', 'ap_417', 'ap_416', 'ap_415', 'ap_414', 'ap_413', 'ap_412', 'ap_441', 'ap_440', 'ap_469', 'ap_442', 'ap_467', 'ap_461', 'ap_462', 'ap_463', 'ap_464', 'ap_465', 'ap_466', 'ap_468', 'ap_443', 'ap_409', 'ap_470', 'ap_471', 'ap_472', 'ap_473', 'ap_474', 'ap_460', 'ap_459', 'ap_458', 'ap_457', 'ap_456', 'ap_455', 'ap_454', 'ap_453', 'ap_452', 'ap_451', 'ap_450', 'ap_449', 'ap_448', 'ap_447', 'ap_446', 'ap_445', 'ap_444', 'ap_410', 'ap_377', 'ap_408', 'ap_357', 'ap_372', 'ap_371', 'ap_370', 'ap_368', 'ap_367', 'ap_366', 'ap_365', 'ap_364', 'ap_363', 'ap_362', 'ap_361', 'ap_360', 'ap_359', 'ap_358', 'ap_356', 'ap_374', 'ap_355', 'ap_354', 'ap_353', 'ap_352', 'ap_351', 'ap_350', 'ap_349', 'ap_348', 'ap_347', 'ap_346', 'ap_345', 'ap_344', 'ap_343', 'ap_341', 'ap_373', 'ap_375', 'ap_407', 'ap_392', 'ap_406', 'ap_405', 'ap_404', 'ap_403', 'ap_402', 'ap_401', 'ap_400', 'ap_399', 'ap_398', 'ap_397', 'ap_396', 'ap_395', 'ap_394', 'ap_393', 'ap_391', 'ap_376', 'ap_390', 'ap_389', 'ap_388', 'ap_387', 'ap_386', 'ap_385', 'ap_384', 'ap_383', 'ap_382', 'ap_381', 'ap_380', 'ap_379', 'ap_378', 'ap_476', 'ap_475', 'ap_509', 'ap_477', 'ap_571', 'ap_565', 'ap_566', 'ap_567', 'ap_568', 'ap_569', 'ap_570', 'ap_572', 'ap_546', 'ap_573', 'ap_574', 'ap_575', 'ap_576', 'ap_577', 'ap_578', 'ap_564', 'ap_563', 'ap_562', 'ap_561', 'ap_560', 'ap_559', 'ap_558', 'ap_557', 'ap_556', 'ap_555', 'ap_554', 'ap_553', 'ap_552', 'ap_551', 'ap_550', 'ap_549', 'ap_548', 'ap_579', 'ap_580', 'ap_581', 'ap_598', 'ap_612', 'ap_611', 'ap_610', 'ap_609', 'ap_608', 'ap_607', 'ap_606', 'ap_605', 'ap_604', 'ap_603', 'ap_602', 'ap_601', 'ap_600', 'ap_599', 'ap_597', 'ap_582', 'ap_596', 'ap_595', 'ap_594', 'ap_593', 'ap_592', 'ap_591', 'ap_590', 'ap_589', 'ap_588', 'ap_587', 'ap_586', 'ap_585', 'ap_584', 'ap_583', 'ap_547', 'ap_545', 'ap_478', 'ap_502', 'ap_496', 'ap_497', 'ap_498', 'ap_499', 'ap_500', 'ap_501', 'ap_503', 'ap_544', 'ap_504', 'ap_505', 'ap_506', 'ap_507', 'ap_508', 'ap_339', 'ap_495', 'ap_494', 'ap_493', 'ap_492', 'ap_491', 'ap_490', 'ap_489', 'ap_488', 'ap_487', 'ap_486', 'ap_485', 'ap_484']


In [ ]:
top_features_400 = ['ap_2', 'ap_1', 'ap_3', 'ap_4', 'ap_5', 'ap_6', 'ap_7', 'ap_8', 'ap_9', 'ap_10', 'ap_11', 'ap_12', 'ap_13', 'ap_14', 'ap_15', 'ap_16', 'ap_17', 'ap_18', 'ap_20', 'ap_19', 'ap_22', 'ap_21', 'ap_27', 'ap_26', 'ap_24', 'ap_31', 'ap_29', 'ap_28', 'ap_23', 'ap_25', 'ap_36', 'ap_32', 'ap_30', 'ap_43', 'ap_42', 'ap_38', 'ap_35', 'ap_34', 'ap_50', 'ap_46', 'ap_47', 'ap_33', 'ap_53', 'ap_57', 'ap_45', 'ap_37', 'ap_52', 'ap_59', 'ap_40', 'ap_58', 'ap_48', 'ap_51', 'ap_70', 'ap_44', 'ap_41', 'ap_54', 'ap_60', 'ap_61', 'ap_65', 'ap_39', 'ap_68', 'ap_56', 'ap_126', 'ap_83', 'ap_139', 'ap_342', 'ap_1161', 'ap_369', 'ap_436', 'ap_437', 'ap_438', 'ap_426', 'ap_435', 'ap_434', 'ap_433', 'ap_432', 'ap_431', 'ap_430', 'ap_429', 'ap_428', 'ap_439', 'ap_427', 'ap_411', 'ap_425', 'ap_424', 'ap_423', 'ap_422', 'ap_421', 'ap_420', 'ap_419', 'ap_418', 'ap_417', 'ap_416', 'ap_415', 'ap_414', 'ap_413', 'ap_412', 'ap_441', 'ap_440', 'ap_469', 'ap_442', 'ap_467', 'ap_461', 'ap_462', 'ap_463', 'ap_464', 'ap_465', 'ap_466', 'ap_468', 'ap_443', 'ap_409', 'ap_470', 'ap_471', 'ap_472', 'ap_473', 'ap_474', 'ap_460', 'ap_459', 'ap_458', 'ap_457', 'ap_456', 'ap_455', 'ap_454', 'ap_453', 'ap_452', 'ap_451', 'ap_450', 'ap_449', 'ap_448', 'ap_447', 'ap_446', 'ap_445', 'ap_444', 'ap_410', 'ap_377', 'ap_408', 'ap_357', 'ap_372', 'ap_371', 'ap_370', 'ap_368', 'ap_367', 'ap_366', 'ap_365', 'ap_364', 'ap_363', 'ap_362', 'ap_361', 'ap_360', 'ap_359', 'ap_358', 'ap_356', 'ap_374', 'ap_355', 'ap_354', 'ap_353', 'ap_352', 'ap_351', 'ap_350', 'ap_349', 'ap_348', 'ap_347', 'ap_346', 'ap_345', 'ap_344', 'ap_343', 'ap_341', 'ap_373', 'ap_375', 'ap_407', 'ap_392', 'ap_406', 'ap_405', 'ap_404', 'ap_403', 'ap_402', 'ap_401', 'ap_400', 'ap_399', 'ap_398', 'ap_397', 'ap_396', 'ap_395', 'ap_394', 'ap_393', 'ap_391', 'ap_376', 'ap_390', 'ap_389', 'ap_388', 'ap_387', 'ap_386', 'ap_385', 'ap_384', 'ap_383', 'ap_382', 'ap_381', 'ap_380', 'ap_379', 'ap_378', 'ap_476', 'ap_475', 'ap_509', 'ap_477', 'ap_571', 'ap_565', 'ap_566', 'ap_567', 'ap_568', 'ap_569', 'ap_570', 'ap_572', 'ap_546', 'ap_573', 'ap_574', 'ap_575', 'ap_576', 'ap_577', 'ap_578', 'ap_564', 'ap_563', 'ap_562', 'ap_561', 'ap_560', 'ap_559', 'ap_558', 'ap_557', 'ap_556', 'ap_555', 'ap_554', 'ap_553', 'ap_552', 'ap_551', 'ap_550', 'ap_549', 'ap_548', 'ap_579', 'ap_580', 'ap_581', 'ap_598', 'ap_612', 'ap_611', 'ap_610', 'ap_609', 'ap_608', 'ap_607', 'ap_606', 'ap_605', 'ap_604', 'ap_603', 'ap_602', 'ap_601', 'ap_600', 'ap_599', 'ap_597', 'ap_582', 'ap_596', 'ap_595', 'ap_594', 'ap_593', 'ap_592', 'ap_591', 'ap_590', 'ap_589', 'ap_588', 'ap_587', 'ap_586', 'ap_585', 'ap_584', 'ap_583', 'ap_547', 'ap_545', 'ap_478', 'ap_502', 'ap_496', 'ap_497', 'ap_498', 'ap_499', 'ap_500', 'ap_501', 'ap_503', 'ap_544', 'ap_504', 'ap_505', 'ap_506', 'ap_507', 'ap_508', 'ap_339', 'ap_495', 'ap_494', 'ap_493', 'ap_492', 'ap_491', 'ap_490', 'ap_489', 'ap_488', 'ap_487', 'ap_486', 'ap_485', 'ap_484', 'ap_483', 'ap_482', 'ap_481', 'ap_480', 'ap_479', 'ap_510', 'ap_511', 'ap_512', 'ap_529', 'ap_543', 'ap_542', 'ap_541', 'ap_540', 'ap_539', 'ap_538', 'ap_537', 'ap_536', 'ap_535', 'ap_534', 'ap_533', 'ap_532', 'ap_531', 'ap_530', 'ap_528', 'ap_513', 'ap_527', 'ap_526', 'ap_525', 'ap_524', 'ap_523', 'ap_522', 'ap_521', 'ap_520', 'ap_519', 'ap_518', 'ap_517', 'ap_516', 'ap_515', 'ap_514', 'ap_340', 'ap_306', 'ap_338', 'ap_158', 'ap_152', 'ap_153', 'ap_154', 'ap_155', 'ap_156', 'ap_157', 'ap_159', 'ap_132', 'ap_160', 'ap_161', 'ap_162', 'ap_163', 'ap_164', 'ap_165', 'ap_151', 'ap_150', 'ap_149', 'ap_148', 'ap_147', 'ap_146', 'ap_145', 'ap_144', 'ap_143', 'ap_142', 'ap_141', 'ap_140', 'ap_138', 'ap_137', 'ap_136', 'ap_135', 'ap_134', 'ap_166', 'ap_167', 'ap_168', 'ap_185', 'ap_199', 'ap_198', 'ap_197', 'ap_196', 'ap_195', 'ap_194', 'ap_193', 'ap_192', 'ap_191', 'ap_190', 'ap_189', 'ap_188', 'ap_187', 'ap_186', 'ap_184', 'ap_169', 'ap_183', 'ap_182', 'ap_181', 'ap_180', 'ap_179', 'ap_178']


In [ ]:
top_features_500 = ['ap_2', 'ap_1', 'ap_3', 'ap_4', 'ap_5', 'ap_6', 'ap_7', 'ap_8', 'ap_9', 'ap_10', 'ap_11', 'ap_12', 'ap_13', 'ap_14', 'ap_15', 'ap_16', 'ap_17', 'ap_18', 'ap_20', 'ap_19', 'ap_22', 'ap_21', 'ap_27', 'ap_26', 'ap_24', 'ap_31', 'ap_29', 'ap_28', 'ap_23', 'ap_25', 'ap_36', 'ap_32', 'ap_30', 'ap_43', 'ap_42', 'ap_38', 'ap_35', 'ap_34', 'ap_50', 'ap_46', 'ap_47', 'ap_33', 'ap_53', 'ap_57', 'ap_45', 'ap_37', 'ap_52', 'ap_59', 'ap_40', 'ap_58', 'ap_48', 'ap_51', 'ap_70', 'ap_44', 'ap_41', 'ap_54', 'ap_60', 'ap_61', 'ap_65', 'ap_39', 'ap_68', 'ap_56', 'ap_126', 'ap_83', 'ap_139', 'ap_342', 'ap_1161', 'ap_369', 'ap_436', 'ap_437', 'ap_438', 'ap_426', 'ap_435', 'ap_434', 'ap_433', 'ap_432', 'ap_431', 'ap_430', 'ap_429', 'ap_428', 'ap_439', 'ap_427', 'ap_411', 'ap_425', 'ap_424', 'ap_423', 'ap_422', 'ap_421', 'ap_420', 'ap_419', 'ap_418', 'ap_417', 'ap_416', 'ap_415', 'ap_414', 'ap_413', 'ap_412', 'ap_441', 'ap_440', 'ap_469', 'ap_442', 'ap_467', 'ap_461', 'ap_462', 'ap_463', 'ap_464', 'ap_465', 'ap_466', 'ap_468', 'ap_443', 'ap_409', 'ap_470', 'ap_471', 'ap_472', 'ap_473', 'ap_474', 'ap_460', 'ap_459', 'ap_458', 'ap_457', 'ap_456', 'ap_455', 'ap_454', 'ap_453', 'ap_452', 'ap_451', 'ap_450', 'ap_449', 'ap_448', 'ap_447', 'ap_446', 'ap_445', 'ap_444', 'ap_410', 'ap_377', 'ap_408', 'ap_357', 'ap_372', 'ap_371', 'ap_370', 'ap_368', 'ap_367', 'ap_366', 'ap_365', 'ap_364', 'ap_363', 'ap_362', 'ap_361', 'ap_360', 'ap_359', 'ap_358', 'ap_356', 'ap_374', 'ap_355', 'ap_354', 'ap_353', 'ap_352', 'ap_351', 'ap_350', 'ap_349', 'ap_348', 'ap_347', 'ap_346', 'ap_345', 'ap_344', 'ap_343', 'ap_341', 'ap_373', 'ap_375', 'ap_407', 'ap_392', 'ap_406', 'ap_405', 'ap_404', 'ap_403', 'ap_402', 'ap_401', 'ap_400', 'ap_399', 'ap_398', 'ap_397', 'ap_396', 'ap_395', 'ap_394', 'ap_393', 'ap_391', 'ap_376', 'ap_390', 'ap_389', 'ap_388', 'ap_387', 'ap_386', 'ap_385', 'ap_384', 'ap_383', 'ap_382', 'ap_381', 'ap_380', 'ap_379', 'ap_378', 'ap_476', 'ap_475', 'ap_509', 'ap_477', 'ap_571', 'ap_565', 'ap_566', 'ap_567', 'ap_568', 'ap_569', 'ap_570', 'ap_572', 'ap_546', 'ap_573', 'ap_574', 'ap_575', 'ap_576', 'ap_577', 'ap_578', 'ap_564', 'ap_563', 'ap_562', 'ap_561', 'ap_560', 'ap_559', 'ap_558', 'ap_557', 'ap_556', 'ap_555', 'ap_554', 'ap_553', 'ap_552', 'ap_551', 'ap_550', 'ap_549', 'ap_548', 'ap_579', 'ap_580', 'ap_581', 'ap_598', 'ap_612', 'ap_611', 'ap_610', 'ap_609', 'ap_608', 'ap_607', 'ap_606', 'ap_605', 'ap_604', 'ap_603', 'ap_602', 'ap_601', 'ap_600', 'ap_599', 'ap_597', 'ap_582', 'ap_596', 'ap_595', 'ap_594', 'ap_593', 'ap_592', 'ap_591', 'ap_590', 'ap_589', 'ap_588', 'ap_587', 'ap_586', 'ap_585', 'ap_584', 'ap_583', 'ap_547', 'ap_545', 'ap_478', 'ap_502', 'ap_496', 'ap_497', 'ap_498', 'ap_499', 'ap_500', 'ap_501', 'ap_503', 'ap_544', 'ap_504', 'ap_505', 'ap_506', 'ap_507', 'ap_508', 'ap_339', 'ap_495', 'ap_494', 'ap_493', 'ap_492', 'ap_491', 'ap_490', 'ap_489', 'ap_488', 'ap_487', 'ap_486', 'ap_485', 'ap_484', 'ap_483', 'ap_482', 'ap_481', 'ap_480', 'ap_479', 'ap_510', 'ap_511', 'ap_512', 'ap_529', 'ap_543', 'ap_542', 'ap_541', 'ap_540', 'ap_539', 'ap_538', 'ap_537', 'ap_536', 'ap_535', 'ap_534', 'ap_533', 'ap_532', 'ap_531', 'ap_530', 'ap_528', 'ap_513', 'ap_527', 'ap_526', 'ap_525', 'ap_524', 'ap_523', 'ap_522', 'ap_521', 'ap_520', 'ap_519', 'ap_518', 'ap_517', 'ap_516', 'ap_515', 'ap_514', 'ap_340', 'ap_306', 'ap_338', 'ap_158', 'ap_152', 'ap_153', 'ap_154', 'ap_155', 'ap_156', 'ap_157', 'ap_159', 'ap_132', 'ap_160', 'ap_161', 'ap_162', 'ap_163', 'ap_164', 'ap_165', 'ap_151', 'ap_150', 'ap_149', 'ap_148', 'ap_147', 'ap_146', 'ap_145', 'ap_144', 'ap_143', 'ap_142', 'ap_141', 'ap_140', 'ap_138', 'ap_137', 'ap_136', 'ap_135', 'ap_134', 'ap_166', 'ap_167', 'ap_168', 'ap_185', 'ap_199', 'ap_198', 'ap_197', 'ap_196', 'ap_195', 'ap_194', 'ap_193', 'ap_192', 'ap_191', 'ap_190', 'ap_189', 'ap_188', 'ap_187', 'ap_186', 'ap_184', 'ap_169', 'ap_183', 'ap_182', 'ap_181', 'ap_180', 'ap_179', 'ap_178', 'ap_177', 'ap_176', 'ap_175', 'ap_174', 'ap_173', 'ap_172', 'ap_171', 'ap_170', 'ap_133', 'ap_131', 'ap_337', 'ap_87', 'ap_80', 'ap_81', 'ap_82', 'ap_84', 'ap_85', 'ap_86', 'ap_88', 'ap_130', 'ap_89', 'ap_90', 'ap_91', 'ap_92', 'ap_93', 'ap_94', 'ap_79', 'ap_78', 'ap_77', 'ap_76', 'ap_75', 'ap_74', 'ap_73', 'ap_72', 'ap_71', 'ap_69', 'ap_67', 'ap_66', 'ap_64', 'ap_63', 'ap_62', 'ap_55', 'ap_49', 'ap_95', 'ap_96', 'ap_97', 'ap_114', 'ap_129', 'ap_128', 'ap_127', 'ap_125', 'ap_124', 'ap_123', 'ap_122', 'ap_121', 'ap_120', 'ap_119', 'ap_118', 'ap_117', 'ap_116', 'ap_115', 'ap_113', 'ap_98', 'ap_112', 'ap_111', 'ap_110', 'ap_109', 'ap_108', 'ap_107', 'ap_106', 'ap_105', 'ap_104', 'ap_103', 'ap_102', 'ap_101', 'ap_100', 'ap_99', 'ap_200', 'ap_201', 'ap_202', 'ap_295', 'ap_289', 'ap_290', 'ap_291', 'ap_292', 'ap_293', 'ap_294', 'ap_296', 'ap_203', 'ap_297', 'ap_298', 'ap_299', 'ap_300', 'ap_301', 'ap_302', 'ap_288', 'ap_287', 'ap_286', 'ap_285', 'ap_284']


In [ ]:
print(X)

In [ ]:
primes = [-301/303, -299/303, -99/101, -295/303, -293/303, -97/101, -289/303, -287/303, -95/101, -283/303, -281/303, -93/101, -277/303, -275/303, -91/101, -271/303, -269/303, -89/101, -265/303, -263/303, -87/101, -259/303, -257/303, -85/101, -253/303, -251/303, -83/101, -247/303, -245/303, -81/101, -241/303, -239/303, -79/101, -235/303, -233/303, -77/101, -229/303, -227/303, -75/101, -223/303, -221/303, -73/101, -217/303, -215/303, -71/101, -211/303, -209/303, -69/101, -205/303, -203/303, -67/101, -199/303, -197/303, -65/101, -193/303, -191/303, -63/101, -187/303, -185/303, -61/101, -181/303, -179/303, -59/101, -175/303, -173/303, -57/101, -169/303, -167/303, -55/101, -163/303, -161/303, -53/101, -157/303, -155/303, -51/101, -151/303, -149/303, -49/101, -145/303, -143/303, -47/101, -139/303, -137/303, -45/101, -133/303, -131/303, -43/101, -127/303, -125/303, -41/101, -121/303, -119/303, -39/101, -115/303, -113/303, -37/101, -109/303, -107/303, -35/101, -103/303, -1/3, -33/101, -97/303, -95/303, -31/101, -91/303, -89/303, -29/101, -85/303, -83/303, -27/101, -79/303, -77/303, -25/101, -73/303, -71/303, -23/101, -67/303, -65/303, -21/101, -61/303, -59/303, -19/101, -55/303, -53/303, -17/101, -49/303, -47/303, -15/101, -43/303, -41/303, -13/101, -37/303, -35/303, -11/101, -31/303, -29/303, -9/101, -25/303, -23/303, -7/101, -19/303, -17/303, -5/101, -13/303, -11/303, -3/101, -7/303, -5/303, -1/101, -1/303, 1/303, 1/101, 5/303, 7/303, 3/101, 11/303, 13/303, 5/101, 17/303, 19/303, 7/101, 23/303, 25/303, 9/101, 29/303, 31/303, 11/101, 35/303, 37/303, 13/101, 41/303, 43/303, 15/101, 47/303, 49/303, 17/101, 53/303, 55/303, 19/101, 59/303, 61/303, 21/101, 65/303, 67/303, 23/101, 71/303, 73/303, 25/101, 77/303, 79/303, 27/101, 83/303, 85/303, 29/101, 89/303, 91/303, 31/101, 95/303, 97/303, 33/101, 1/3, 103/303, 35/101, 107/303, 109/303, 37/101, 113/303, 115/303, 39/101, 119/303, 121/303, 41/101, 125/303, 127/303, 43/101, 131/303, 133/303, 45/101, 137/303, 139/303, 47/101, 143/303, 145/303, 49/101, 149/303, 151/303, 51/101, 155/303, 157/303, 53/101, 161/303, 163/303, 55/101, 167/303, 169/303, 57/101, 173/303, 175/303, 59/101, 179/303, 181/303, 61/101, 185/303, 187/303, 63/101, 191/303, 193/303, 65/101, 197/303, 199/303, 67/101, 203/303, 205/303, 69/101, 209/303, 211/303, 71/101, 215/303, 217/303, 73/101, 221/303, 223/303, 75/101, 227/303, 229/303, 77/101, 233/303, 235/303, 79/101, 239/303, 241/303, 81/101, 245/303, 247/303, 83/101, 251/303, 253/303, 85/101, 257/303, 259/303, 87/101, 263/303, 265/303, 89/101, 269/303, 271/303, 91/101, 275/303, 277/303, 93/101, 281/303, 283/303, 95/101, 287/303, 289/303, 97/101, 293/303, 295/303, 99/101, 299/303, 301/303, 1]
abs_prime = [abs(prime) for prime in primes]

In [ ]:
for i in range(min(len(abs_prime),97)):
    ap_col = f"ap_{i+1}"
    df_sub[ap_col].dropna(inplace=True)
    if (df_sub[ap_col] == 0).any():
        df_sub[ap_col].replace(0, df_sub[ap_col].mean(), inplace=True)
    df_sub[ap_col] = df_sub[ap_col].apply(lambda x, p=abs_prime[i]: x / math.sqrt(p))

In [ ]:
X = df_sub[['root']+ top_features_200]
y = df_sub['rank']

In [ ]:
feature_col = [col for col in df.columns if col.startswith('ap_')]
X = df_sub[['root']+ feature_col]
y = df_sub['rank']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.6, random_state=42)
X_train_val, X_test_val, y_train_val, y_test_val = train_test_split(X_train, y_train, test_size=0.5, random_state=42)

In [ ]:
import lightgbm as lgb

clf = lgb.LGBMClassifier(n_estimators=500,max_depth=50,learning_rate= 0.1)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test_val)

print("RMSE: {:.4f}".format(np.sqrt(mean_squared_error(y_test_val, y_pred))))
print("R^2 Score: {:.4f}".format(r2_score(y_test_val, y_pred)))
print("ACC Score: {:.4f}".format(accuracy_score(y_test_val, y_pred)))

RMSE: 0.1921
R^2 Score: 0.9705
ACC Score: 0.9820


In [ ]:
import lightgbm as lgb
#200
clf = lgb.LGBMClassifier(n_estimators=100,max_depth=50,learning_rate= 0.1)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test_val)

print("RMSE: {:.4f}".format(np.sqrt(mean_squared_error(y_test_val, y_pred))))
print("R^2 Score: {:.4f}".format(r2_score(y_test_val, y_pred)))
print("ACC Score: {:.4f}".format(accuracy_score(y_test_val, y_pred)))

RMSE: 0.6320
R^2 Score: 0.6819
ACC Score: 0.8166


In [ ]:
feature_col = [col for col in df.columns if col.startswith('ap_')]
X = df_sub[ feature_col]
y = df_sub['rank']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.6, random_state=42)
X_train_val, X_test_val, y_train_val, y_test_val = train_test_split(X_train, y_train, test_size=0.5, random_state=42)

In [ ]:
import lightgbm as lgb

clf = lgb.LGBMClassifier(n_estimators=100,max_depth=50,learning_rate= 0.1)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test_val)

print("RMSE: {:.4f}".format(np.sqrt(mean_squared_error(y_test_val, y_pred))))
print("R^2 Score: {:.4f}".format(r2_score(y_test_val, y_pred)))
print("ACC Score: {:.4f}".format(accuracy_score(y_test_val, y_pred)))

RMSE: 0.4941
R^2 Score: 0.8056
ACC Score: 0.9018


In [ ]:
n_clusters = 4
kmeans = KMeans(n_clusters=n_clusters)
kmeans.fit(X)
df_sub['cluster_label'] = kmeans.labels_

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
X_reduced = df_sub[['cluster_label']+['root'] + feature_col]
y_reduced = df_sub['rank']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_reduced, y_reduced, test_size=0.6, random_state=10)
X_train_val, X_test_val, y_train_val, y_test_val = train_test_split(X_train, y_train, test_size=0.5, random_state=42)

In [ ]:
import xgboost as xgb

dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_train_val, label=y_train_val)

params = {
    'objective': 'multi:softprob',
    'num_class': 4,
    'eval_metric': 'auc',
    'max_depth': 4,
    'learning_rate': 0.05,
    'subsample': 0.6,
    'colsample_bytree': 0.6,
    'reg_alpha': 0.4,
    'reg_lambda': 0.4,
    'random_state': 42
}


xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round=500,
    evals=[(dtrain, 'train'), (dval, 'validation')],
    early_stopping_rounds=10,
    verbose_eval=10
)
dtest = xgb.DMatrix(X_test)
ytest = xgb.DMatrix(y_test)
test_pro = xgb_model.predict(dtest)
test_pre = np.argmax(test_pro, axis=1)



test_accuracy = np.sum(test_pre == y_test) / len(y_test)
print("Test Accuracy:", test_accuracy)


dtest_val = xgb.DMatrix(X_test_val)
ytest_val = xgb.DMatrix(y_test_val)

test_pro_val = xgb_model.predict(dtest_val)
test_pre_val = np.argmax(test_pro_val, axis=1)



test_accuracy = np.sum(test_pre_val == y_test_val) / len(y_test_val)
print("val Accuracy:", test_accuracy)

[0]	train-auc:0.64117	validation-auc:0.64051
[10]	train-auc:0.70362	validation-auc:0.70240
[20]	train-auc:0.71092	validation-auc:0.70939
[30]	train-auc:0.71865	validation-auc:0.71736
[40]	train-auc:0.72535	validation-auc:0.72407
[50]	train-auc:0.73095	validation-auc:0.72960
[60]	train-auc:0.73714	validation-auc:0.73585
[70]	train-auc:0.74446	validation-auc:0.74319
[80]	train-auc:0.75013	validation-auc:0.74888
[90]	train-auc:0.75599	validation-auc:0.75467
[100]	train-auc:0.76149	validation-auc:0.76028
[110]	train-auc:0.76708	validation-auc:0.76586
[120]	train-auc:0.77230	validation-auc:0.77102
[130]	train-auc:0.77696	validation-auc:0.77562
[140]	train-auc:0.78178	validation-auc:0.78049
[150]	train-auc:0.78609	validation-auc:0.78470
[160]	train-auc:0.79041	validation-auc:0.78910
[170]	train-auc:0.79414	validation-auc:0.79272
[180]	train-auc:0.79783	validation-auc:0.79646
[190]	train-auc:0.80136	validation-auc:0.79992
[200]	train-auc:0.80498	validation-auc:0.80348
[210]	train-auc:0.80796	

In [ ]:
import xgboost as xgb

dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_train_val, label=y_train_val)

params = {
    'objective': 'multi:softprob',
    'num_class': 4,
    'eval_metric': 'auc',
    'max_depth': 4,
    'learning_rate': 0.1,
    'subsample': 0.6,
    'colsample_bytree': 0.8,
    'reg_alpha': 0.5,
    'reg_lambda': 0.5,
    'random_state': 42
}


xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round=200,
    evals=[(dtrain, 'train'), (dval, 'validation')],
    early_stopping_rounds=10,
    verbose_eval=10
)
dtest = xgb.DMatrix(X_test)
ytest = xgb.DMatrix(y_test)
test_pro = xgb_model.predict(dtest)
test_pre = np.argmax(test_pro, axis=1)



test_accuracy = np.sum(test_pre == y_test) / len(y_test)
print("Test Accuracy:", test_accuracy)


dtest_val = xgb.DMatrix(X_test_val)
ytest_val = xgb.DMatrix(y_test_val)

test_pro_val = xgb_model.predict(dtest_val)
test_pre_val = np.argmax(test_pro_val, axis=1)



test_accuracy = np.sum(test_pre_val == y_test_val) / len(y_test_val)
print("val Accuracy:", test_accuracy)

[0]	train-auc:0.66163	validation-auc:0.66307
[10]	train-auc:0.70044	validation-auc:0.70155
[20]	train-auc:0.72008	validation-auc:0.72126
[30]	train-auc:0.73522	validation-auc:0.73624
[40]	train-auc:0.74880	validation-auc:0.74971
[50]	train-auc:0.76040	validation-auc:0.76140
[60]	train-auc:0.77166	validation-auc:0.77246
[70]	train-auc:0.78071	validation-auc:0.78146
[80]	train-auc:0.78924	validation-auc:0.78962
[90]	train-auc:0.79642	validation-auc:0.79666
[100]	train-auc:0.80326	validation-auc:0.80349
[110]	train-auc:0.80932	validation-auc:0.80956
[120]	train-auc:0.81544	validation-auc:0.81559
[130]	train-auc:0.82093	validation-auc:0.82099
[140]	train-auc:0.82639	validation-auc:0.82636
[150]	train-auc:0.83158	validation-auc:0.83149
[160]	train-auc:0.83632	validation-auc:0.83607
[170]	train-auc:0.84042	validation-auc:0.84020
[180]	train-auc:0.84472	validation-auc:0.84443
[190]	train-auc:0.84868	validation-auc:0.84844
[199]	train-auc:0.85189	validation-auc:0.85165
Test Accuracy: 0.4752222

In [ ]:
import xgboost as xgb

xgb_model = xgb.XGBClassifier()
xgb_model.fit(X_train, y_train)

val_predictions = xgb_model.predict(X_test_val)
val_accuracy = np.sum(val_predictions == y_test_val) / len(y_test_val)
print("Validation Accuracy:", val_accuracy)

test_predictions = xgb_model.predict(X_test)
test_accuracy = np.sum(test_predictions == y_test) / len(y_test)
print("Test Accuracy:", test_accuracy)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_reduced)

In [ ]:
# Plotting the clusters
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=kmeans.labels_, cmap='viridis')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('K-means Clustering (PCA)')
plt.show()